In [1]:
import sqlite3

# connect (creates file if it doesn't exist)
conn = sqlite3.connect(":memory:")
cur = conn.cursor()

# make sure FK constraints are enforced
cur.execute("PRAGMA foreign_keys = ON;")

# ----- base tables -----

cur.execute("""
CREATE TABLE department (
    dept_name   TEXT PRIMARY KEY,
    building    TEXT,
    budget      NUMERIC
);
""")

cur.execute("""
CREATE TABLE classroom (
    building    TEXT,
    room_number TEXT,
    capacity    INTEGER,
    PRIMARY KEY (building, room_number)
);
""")

cur.execute("""
CREATE TABLE time_slot (
    time_slot_id TEXT PRIMARY KEY,
    day          TEXT,
    start_hr     INTEGER,
    start_min    INTEGER,
    end_hr       INTEGER,
    end_min      INTEGER
);
""")

cur.execute("""
CREATE TABLE course (
    course_id  TEXT PRIMARY KEY,
    title      TEXT,
    dept_name  TEXT,
    credits    INTEGER,
    FOREIGN KEY (dept_name) REFERENCES department(dept_name)
);
""")

cur.execute("""
CREATE TABLE student (
    ID        TEXT PRIMARY KEY,
    name      TEXT,
    dept_name TEXT,
    tot_cred  INTEGER,
    FOREIGN KEY (dept_name) REFERENCES department(dept_name)
);
""")

cur.execute("""
CREATE TABLE instructor (
    ID        TEXT PRIMARY KEY,
    name      TEXT,
    dept_name TEXT,
    salary    NUMERIC,
    FOREIGN KEY (dept_name) REFERENCES department(dept_name)
);
""")

# ----- relationship tables -----

cur.execute("""
CREATE TABLE advisor (
    s_ID  TEXT PRIMARY KEY,
    i_ID  TEXT,
    FOREIGN KEY (s_ID) REFERENCES student(ID),
    FOREIGN KEY (i_ID) REFERENCES instructor(ID)
);
""")

cur.execute("""
CREATE TABLE section (
    course_id   TEXT,
    sec_id      TEXT,
    semester    TEXT,
    year        INTEGER,
    building    TEXT,
    room_number TEXT,
    time_slot_id TEXT,
    PRIMARY KEY (course_id, sec_id, semester, year),
    FOREIGN KEY (course_id) REFERENCES course(course_id),
    FOREIGN KEY (building, room_number)
        REFERENCES classroom(building, room_number),
    FOREIGN KEY (time_slot_id) REFERENCES time_slot(time_slot_id)
);
""")

cur.execute("""
CREATE TABLE takes (
    ID        TEXT,
    course_id TEXT,
    sec_id    TEXT,
    semester  TEXT,
    year      INTEGER,
    grade     TEXT,
    PRIMARY KEY (ID, course_id, sec_id, semester, year),
    FOREIGN KEY (ID) REFERENCES student(ID),
    FOREIGN KEY (course_id, sec_id, semester, year)
        REFERENCES section(course_id, sec_id, semester, year)
);
""")

cur.execute("""
CREATE TABLE teaches (
    ID        TEXT,
    course_id TEXT,
    sec_id    TEXT,
    semester  TEXT,
    year      INTEGER,
    PRIMARY KEY (ID, course_id, sec_id, semester, year),
    FOREIGN KEY (ID) REFERENCES instructor(ID),
    FOREIGN KEY (course_id, sec_id, semester, year)
        REFERENCES section(course_id, sec_id, semester, year)
);
""")

cur.execute("""
CREATE TABLE prereq (
    course_id  TEXT,
    prereq_id  TEXT,
    PRIMARY KEY (course_id, prereq_id),
    FOREIGN KEY (course_id) REFERENCES course(course_id),
    FOREIGN KEY (prereq_id) REFERENCES course(course_id)
);
""")


In [3]:
# Cell: Insert dummy data into all tables
# Inserts follow FK order: department -> classroom/time_slot -> course -> student/instructor -> section -> takes/teaches/advisor/prereq
departments = [
    ('Computer Science','CS Building',120000),
    ('Mathematics','Math Hall',80000),
    ('Physics','Physics Tower',90000)
]
cur.executemany('INSERT OR REPLACE INTO department (dept_name, building, budget) VALUES (?,?,?)', departments)

classrooms = [
    ('CS Building','101',50),
    ('CS Building','102',30),
    ('Math Hall','201',40)
]
cur.executemany('INSERT OR REPLACE INTO classroom (building, room_number, capacity) VALUES (?,?,?)', classrooms)

time_slots = [
    ('TS1','M',9,0,9,50),
    ('TS2','W',10,0,10,50),
    ('TS3','F',11,0,11,50)
]
cur.executemany('INSERT OR REPLACE INTO time_slot (time_slot_id, day, start_hr, start_min, end_hr, end_min) VALUES (?,?,?,?,?,?)', time_slots)

courses = [
    ('CS101','Intro to CS','Computer Science',3),
    ('CS201','Data Structures','Computer Science',3),
    ('MATH101','Calculus I','Mathematics',4),
    ('PHYS101','Physics I','Physics',4)
]
cur.executemany('INSERT OR REPLACE INTO course (course_id, title, dept_name, credits) VALUES (?,?,?,?)', courses)

students = [
    ('10001','Alice','Computer Science',15),
    ('10002','Bob','Mathematics',32),
    ('10003','Cathy','Physics',45),
    ('10004','Dan','Computer Science',62),
    ('10005','Eve','Mathematics',75),
    ('10006','Frank','Physics',88),
    ('10007','Grace','Computer Science',95),
    ('10008','Heidi','Mathematics',110),
    ('10009','Ivan','Physics',121)
]
cur.executemany('INSERT OR REPLACE INTO student (ID, name, dept_name, tot_cred) VALUES (?,?,?,?)', students)

instructors = [
    ('20001','Prof. X','Computer Science',90000),
    ('20002','Dr. Y','Mathematics',80000),
    ('20003','Dr. Z','Physics',85000)
]
cur.executemany('INSERT OR REPLACE INTO instructor (ID, name, dept_name, salary) VALUES (?,?,?,?)', instructors)

advisors = [
    ('10001','20001'),
    ('10002','20002'),
    ('10003','20003'),
    ('10004','20001')
]
cur.executemany('INSERT OR REPLACE INTO advisor (s_ID, i_ID) VALUES (?,?)', advisors)

sections = [
    ('CS101','S1','Fall',2025,'CS Building','101','TS1'),
    ('CS201','S1','Fall',2025,'CS Building','102','TS2'),
    ('MATH101','S1','Fall',2025,'Math Hall','201','TS1'),
    ('PHYS101','S1','Fall',2025,'CS Building','101','TS3')
]
cur.executemany('INSERT OR REPLACE INTO section (course_id, sec_id, semester, year, building, room_number, time_slot_id) VALUES (?,?,?,?,?,?,?)', sections)

takes = [
    ('10001','CS101','S1','Fall',2025,'A'),
    ('10002','MATH101','S1','Fall',2025,'B'),
    ('10003','PHYS101','S1','Fall',2025,None),
    ('10004','CS201','S1','Fall',2025,'B')
]
cur.executemany('INSERT OR REPLACE INTO takes (ID, course_id, sec_id, semester, year, grade) VALUES (?,?,?,?,?,?)', takes)

teaches = [
    ('20001','CS101','S1','Fall',2025),
    ('20001','CS201','S1','Fall',2025),
    ('20002','MATH101','S1','Fall',2025),
    ('20003','PHYS101','S1','Fall',2025)
]
cur.executemany('INSERT OR REPLACE INTO teaches (ID, course_id, sec_id, semester, year) VALUES (?,?,?,?,?)', teaches)

prereqs = [
    ('CS201','CS101'),
    ('PHYS101','MATH101')
]
cur.executemany('INSERT OR REPLACE INTO prereq (course_id, prereq_id) VALUES (?,?)', prereqs)

conn.commit()
print('Dummy data inserted: departments={}, classrooms={}, time_slots={}, courses={}, students={}, instructors={}, advisors={}, sections={}, takes={}, teaches={}, prereqs={}'.format(len(departments), len(classrooms), len(time_slots), len(courses), len(students), len(instructors), len(advisors), len(sections), len(takes), len(teaches), len(prereqs)))

Dummy data inserted: departments=3, classrooms=3, time_slots=3, courses=4, students=9, instructors=3, advisors=4, sections=4, takes=4, teaches=4, prereqs=2
